## Imports and setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from stat_funcs import movement_likelihood_func, new_or_gone_likelihood_func_Y, multi_bubble_likelihood_func
from associator import Associator, asc_condition, comb_constr
from trajectories import node_trajectory_with_stats
from optimizer import optimizer
from tracer import Tracer

import glob
import os

plt.rcParams['figure.dpi'] = 500
np.set_printoptions(suppress=True)

## Locating all sequences

In [2]:
Drive = 'C:\\'
W_dir = Drive + os.path.join(*(os.getcwd().split('\\')[1:-1] + ['Objects']))
os.chdir(W_dir)
main_dirs = sorted(glob.glob('./*'))
for i, x in enumerate(main_dirs): print(i, ':',x)

0 : .\Area_Output - 120 1.0 T
1 : .\Area_Output - 120 1.4 T
2 : .\Area_Output - 2D Laminar Field OFF 2D_lm_animation_noField_100
3 : .\Area_Output - 2D Laminar Field OFF 2D_lm_animation_noField_120
4 : .\Empty_directory_tree
5 : .\Field OFF 100 sccm Round I
6 : .\NEUTRONS Field OFF 100 sccm
7 : .\NEUTRONS Field OFF 160 sccm
8 : .\NEUTRONS Field ON 100 sccm
9 : .\NEUTRONS Field ON 160 sccm
10 : .\Particles
11 : .\Round I Field OFF 140 sccm
12 : .\Round I Field OFF 160 sccm v2
13 : .\Round I Field ON 100 sccm
14 : .\Round III Field OFF 120 sccm
15 : .\Round III Field OFF 120 sccm v2
16 : .\Round III HFS 125 mT 120 sccm
17 : .\Round III VFS 125 mT 120 sccm
18 : .\desktop.ini


## Selecting a sequence

In [3]:
I = 17
J = 0

Main_dir = main_dirs[I]
sub_dirs = glob.glob(Main_dir + '/*')
try: sub_dirs.remove(*glob.glob(Main_dir + '/**.ini'))
except: pass
Sub_dir  = sub_dirs[J]
print(Sub_dir)

.\Round III VFS 125 mT 120 sccm\Front view


In [4]:
#@title Statistical function setup

Sig_displacement1   = 50  #@param {type: "slider", min: 10, max: 100}
K1                  = 0.3 #@param {type:"slider", min:0, max:1, step:0.01}
Move   = movement_likelihood_func(Sig_displacement1, K1)

A                   = 0.1 #@param {type:"slider", min:0.01, max:0.5, step:0.01}
Boundary            = 20 #@param {type:"slider", min:0, max:50}
Height              = 919 #@param {type:"slider", min:0, max:1500}
New    = new_or_gone_likelihood_func_Y(A, Boundary, 1)
Gone   = new_or_gone_likelihood_func_Y(-A, Height - Boundary, 0)

Sig_displacement2   = 64 #@param {type:"slider", min:0, max:150}
K2                  = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
Merge  = multi_bubble_likelihood_func(Sig_displacement2, K2, 0)
Split  = multi_bubble_likelihood_func(Sig_displacement2, K2, 1)

Optimizer     = optimizer([Move, New, Gone, Merge, Split])

## Associator setup

In [5]:
#@title Associator setup

Max_displacement_per_frame  = 300  #@param {type: "slider", min: 10, max: 500}
Radius_multiplyer           = 4 #@param {type:"slider", min:1, max:10}
Min_displacement            = 30 #@param {type:"slider", min:0, max:100}
Asc_condition  = asc_condition(Max_displacement_per_frame, Radius_multiplyer, Min_displacement)

Upsilon                     = 1.5 #@param {type:"slider", min:0.01, max:1.5, step:0.01}
Mu_v                        = 300 #@param {type:"slider", min:0, max:300}
Max_acc                     = 132 #@param {type:"slider", min:0, max:300}
Comb_constr = comb_constr(Upsilon, Mu_v, Max_acc)

ASSociator = Associator(Asc_condition, Comb_constr)

In [6]:
#@title Trajectory setup

mu_V       = 20 #@param {type:"slider", min:0, max:100}
sig_V      = 30 #@param {type:"slider", min:0, max:100}
r_sig_S    = 1.5 #@param {type:"slider", min:0.01, max:1.5, step:0.01}
node_trajectory = node_trajectory_with_stats(mu_V, sig_V, r_sig_S)

## Tracing

In [7]:
Max_occlusion = 3
Quantile = 0.05
tracer = Tracer(ASSociator, Optimizer, node_trajectory, Max_occlusion, Quantile, Sub_dir)

Window width - 1:   0%|                                                               | 2/2999 [00:00<03:32, 14.14it/s]

Trajectory count :288


Window width - 2:   0%|                                                               | 2/2999 [00:00<03:29, 14.32it/s]

Trajectory count :288


Window width - 3:   0%|                                                               | 2/2999 [00:00<03:07, 16.02it/s]

Trajectory count :199


Window width - 3: 100%|████████████████████████████████████████████████████████████| 2999/2999 [03:27<00:00, 14.47it/s]


Trajectory count :170


## Data dumping

In [8]:
Indx = 7 #@param {type: "slider", min: 1, max: 100}
Dump_site = '/' + 'test_%i_'%Indx +str(Max_occlusion)

Memory = 15
Min_trajctory_lenght = 1

tracer.dump_data(Dump_site, Memory, Min_trajctory_lenght)

Loading images : 100%|█████████████████████████████████████████████████████████████| 3000/3000 [00:56<00:00, 52.92it/s]
Drawing trajectories : 170it [00:02, 78.11it/s]
Saving: trajectories: 170it [00:00, 280.66it/s]
Drawing families : 166it [00:02, 77.60it/s]
Saving: families: 166it [00:00, 197.25it/s]
Drawing ID history : 170it [00:00, 557.24it/s]
Saving: tracedIDs: 3000it [00:28, 105.24it/s]
Drawing velocity history : 170it [00:00, 287.08it/s]
Saving: traced_velocities: 3000it [00:22, 131.30it/s]
